In [0]:
from fastai.text import * 

In [2]:
path = '/content/drive/My Drive/Vaccine Hackathon'
path

'/content/drive/My Drive/Vaccine Hackathon'

In [3]:
df = pd.read_csv('/content/drive/My Drive/Vaccine Hackathon/Train.csv')
df.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [0]:
df = df.drop('tweet_id', axis = 1)

In [5]:
df

,safe_text,label,agreement
0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.000000
1,I'm 100% thinking of devoting my career to pro...,1.0,1.000000
2,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.000000
3,I mean if they immunize my kid with something ...,-1.0,1.000000
4,Thanks to <user> Catch me performing at La Nui...,0.0,1.000000
...,...,...,...
9996,Living in a time where the sperm I used to was...,1.0,1.000000
9997,<user> <user> In spite of all measles outbrea...,1.0,0.666667
9998,Interesting trends in child immunization in Ok...,0.0,1.000000
9999,CDC Says Measles Are At Highest Levels In Deca...,0.0,1.000000


In [0]:
order = ['label','safe_text','agreement']


In [0]:
train = df.reindex(columns= order,fill_value=0.5)

In [8]:
train

,label,safe_text,agreement
0,0.0,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,1.000000
1,1.0,I'm 100% thinking of devoting my career to pro...,1.000000
2,-1.0,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",1.000000
3,-1.0,I mean if they immunize my kid with something ...,1.000000
4,0.0,Thanks to <user> Catch me performing at La Nui...,1.000000
...,...,...,...
9996,1.0,Living in a time where the sperm I used to was...,1.000000
9997,1.0,<user> <user> In spite of all measles outbrea...,0.666667
9998,0.0,Interesting trends in child immunization in Ok...,1.000000
9999,0.0,CDC Says Measles Are At Highest Levels In Deca...,1.000000


In [0]:
# get a list of the columns
col_list = list(df)
# use this handy way to swap the elements
col_list[0], col_list[1] = col_list[1], col_list[0]
# assign back, the order will now be swapped
df.columns = col_list

In [0]:
train['label'].fillna(train['label'].mean(), inplace=True)

In [0]:
train.to_csv(r'/content/drive/My Drive/Vaccine Hackathon/train.csv', index = False)

In [10]:
# Language model data
data_lm = TextLMDataBunch.from_csv(path, 'train.csv')
# Classifier model data
data_clas = TextClasDataBunch.from_csv(path, 'train.csv', vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)

In [13]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.665613,3.819398,0.312240,09:35


In [14]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.683380,3.463768,0.365160,10:24


In [15]:
learn.predict(train['safe_text'][0:5], n_words=10)

0    Me &amp; The Big Homie meanboy3000 #MEANBOY #M...
1    I'm 100% thinking of devoting my career to pro...
2    #whatcausesautism VACCINES, DO NOT VACCINATE Y...
3    I mean if they immunize my kid with something ...
4    Thanks to <user> Catch me performing at La Nui...
Name: safe_text, dtype: object

In [0]:
learn.save_encoder('ft_enc')

In [19]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (8000 items)
x: TextList
xxbos “ < user > xxmaj anti - vaccination hysteria , based on lies , led to xxunk outcome : disease outbreak . < url > hippies,xxbos " < user > xxup hiv xxmaj vaccine xxmaj safe xxmaj enough xxmaj to xxmaj pass xxmaj phase 1 xxmaj human xxmaj trials < url > 
  xxmaj dam , getting somewhere,xxbos < user > # xxup gop xxup stop blaming # xxmaj immigrants first # xxmaj ebola now # measles xxmaj which diseases?brought their ancestors ? # xxup ainf < url >,xxbos < user > no mmr xxrep 4 ! # xxunk,xxbos xxmaj let it rain over me < user > # dc # djflyer # dj # instadc # flyer # washingtondc # dmv # mmr # follow … < url >
y: FloatList
1.0,1.0,0.0,0.0,0.0
Path: /content/drive/My Drive/Vaccine Hackathon;

Valid: LabelList (2001 items)
x: TextList
xxbos xxmaj record outbreak of measles in the xxup u.s. , xxup cdc says - xxmaj anti - vaccine xxunk to blame ! < url >,xxbos i just woke up from a measles / xxmaj miley xxmaj c

In [20]:
data_clas.show_batch()

text,target
"xxbos xxup am - xxmaj news : xxmaj week xxmaj ahead : xxup xxunk in the xxup er , xxup hiv xxmaj news , xxmaj docs on xxmaj measles xxmaj vaccine : xxmaj week xxmaj ahead : xxup xxunk in the xxup er , xxup hiv xxmaj news , xxmaj docs … < url >",0.0
"xxbos "" < user > < user > # xxmaj life # xxup safe # xxmaj admitted 98 xxmaj mil # xxup us xxmaj received # xxmaj polio # xxmaj vaccine xxmaj in xxunk xxmaj xxunk xxmaj xxunk xxmaj with # xxmaj cancer # xxmaj virus < user >",-1.0
xxbos ( ( xxmaj silvito xxmaj el xxmaj libre ) ) xxmaj measles is xxmaj not a xxmaj mild xxmaj disease : xxmaj measles xxmaj is xxmaj not a xxmaj mild xxmaj disease i had measles when i was eight years old . xxmaj in those ...,0.0
"xxbos xxmaj diseases more dangerous than ebola : ( xxmaj all xxmaj airborne ) , xxmaj measles , xxmaj sars , xxmaj mers , xxup tb , xxmaj flu , pneumonia , xxmaj whooping xxmaj cough . xxmaj there are also non airborne ➡ ➡",0.0
"xxbos xxmaj reagan 's school sent flyer offering flu vaccines . xxmaj nice . xxmaj no insurance , cost is $ xxunk . xxmaj with insurance , cost is $ 15 . xxup tn care it 's $ 0 . xxmaj say what ?",1.0


In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,time
0,0.384392,0.365082,14:28


In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,time
0,0.364015,0.338942,15:58


In [22]:
learn.predict(data_lm)

(FloatItem [0.325307], tensor([0.3253]), tensor([0.3253]))

In [0]:
test = pd.read_csv('/content/drive/My Drive/Vaccine Hackathon/Test.csv')

In [24]:
test.shape

(5177, 2)

In [0]:
model = learn.predict(data_clas, test['safe_text'].all())

In [33]:
Sample_submission = pd.read_csv('/content/drive/My Drive/Vaccine Hackathon/SampleSubmission.csv', header='infer')
Sample_submission.head()

,tweet_id,label
0,00BHHHP1,0
1,00UNMD0E,0
2,01AXPTJF,0
3,01HOEQJW,0
4,01JUKMAO,0


In [0]:
sample_submission["label"] = model.predict(test['safe_text'].all())

In [0]:
sample_submission.to_csv('Submission2.csv', index= False)